In [13]:
import Minkowski as mn
import AdS as ads
from mm_estimator import mm_dimension
from longest_chain import *
from interval import filter_interval
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as op
%matplotlib tk
from importlib import reload

params= {
   'axes.labelsize': 16,
   'font.size': 18,
   'legend.fontsize': 18,
   'xtick.labelsize': 16,
   'ytick.labelsize': 16,
   'figure.figsize': [8, 6] 
   }
plt.rcParams.update(params)

def scatter(position,x=0,y=1):
    plt.scatter(position[:,x],position[:,y],s=1)

def plot_interval(l,interval_centre,half=True):
   R=interval_centre[0]
   t=interval_centre[1]
   if half==True:
      plt.plot([R,R+l/2,R],[t+l/2,t,t-l/2])
   else:
      plt.plot([R,R+l/2,R,R-l/2,R],[t+l/2,t,t-l/2,t,t+l/2])

def plot_link(position,link,xindex=0,yindex=1):
    for i in link:
        for j in link[i]:
            plt.plot([position[i,xindex],position[j,xindex]],[position[i,yindex],position[j,yindex]],color='b')

def fitting_func(Nintervallist,md,d):
    y=[]
    for i in Nintervallist:
        y.append(md*i**(1/d))
    return y

In [76]:
#plotting links on minkowski space
N=50
L=1
d=2
interval_centre=np.zeros(d)
position=mn.sprinkle(N,L,d,False)
link=find_link(position)
scatter(position)
plot_link(position,link)
plt.title('Sprinkling into Minkowski space with links plotted on at N=%s L=%s d=%s'%(N,L,d),fontsize=13)
plt.xlabel('R')
plt.ylabel('t')
plt.show()

In [34]:
#testing mm estimator
N=1000
L=1
d=2
interval_centre=np.zeros(d)
position=mn.sprinkle(N,L,d,False)
position_subset=filter_interval(position,L,interval_centre)
relation=find_relation(position_subset)
Nrel=sum(len(relation[i]) for i in range(len(relation)))
frac=Nrel/len(position_subset)**2
dim=mm_dimension(frac)
print(dim)

2.0338322002372573


In [57]:
#Sprinkle into AdS space
N=5000
L=1
d=2
R_0=0.1
interval_centre=[R_0,0]
for i in range(d-2):
    interval_centre.append(0)
position=ads.sprinkle(N,L,d,R_0,False,True)
position_subset=filter_interval(position,L,interval_centre)
plt.title(r'Sprinkling into AdS space at N=%s L=%s d=%s $R_0$=%s' %(N,L,d,R_0),fontsize=15)
plt.xlabel('R')
plt.ylabel('t')
scatter(position)
scatter(position_subset)
plot_interval(L,interval_centre,True)
plt.show()

In [24]:
#investigate full interval as function of R_0
N=20000
L=2000
d=2
R_0list=np.linspace(0.01,1,20)

discretenesslist=[]
dimlist=[]
dimstd=[]
for R_0 in R_0list:
    interval_centre=[R_0+L/2,0]
    for i in range(d-2):
        interval_centre.append(0)

    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L/2)**(1-d))
    densityads=N/V
    discreteness=densityads**(-1/d)
    discretenesslist.append(discreteness)

    Nrelsub=[]
    Nintervalsub=[]
    for count1 in range(20):
        position=ads.sprinkle(N,L,d,R_0,False,False)
        position_subset=filter_interval(position,L,interval_centre)
        relation=find_relation(position_subset)
        Nrel=sum(len(relation[i]) for i in range(len(relation)))
        Ninterval=len(position_subset)
        Nrelsub.append(Nrel)
        Nintervalsub.append(Ninterval)
        print(count1)
    Nrelmean=np.mean(Nrelsub)
    Nrelstd=np.std(Nrelsub)
    Nintervalmean=np.mean(Nintervalsub)
    Nintervalstd=np.std(Nintervalsub)
    fracmean=Nrelmean/Nintervalmean**2
    fracstd=np.sqrt((Nrelstd/Nintervalmean**2)**2+(Nintervalstd*2*Nrelmean/Nintervalmean**3)**2)
    upperdim=mm_dimension(fracmean+fracstd)
    lowerdim=mm_dimension(fracmean-fracstd)
    dimlist.append(np.mean([upperdim,lowerdim]))
    dimstd.append(np.std([upperdim,lowerdim]))
    print(R_0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.01
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.06210526315789474
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.11421052631578947
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.16631578947368422
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.21842105263157896
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.2705263157894737
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.32263157894736844
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.37473684210526315
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.4268421052631579
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.4789473684210527
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.5310526315789474
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.5831578947368421
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.6352631578947369
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.6873684210526316
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1

In [26]:
plt.title(r'Discreteness length for interval at $R=R_0+L/2$;N=%s d=%s L=%s'%(N,d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel(r'$l_{discrete}$')
plt.plot(R_0list,discretenesslist)
plt.grid()
plt.show()

In [25]:
plt.title('Myrheim-Meyer dimension for interval at $R=R_0+L/2$;N=%s d=%s L=%s'%(N,d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel('Myrheim-Meyer dimension')
plt.errorbar(R_0list,dimlist,yerr=dimstd)
plt.grid()
plt.show()

In [27]:
#investigate full interval as function of R_0
N=2000
L=1
d=2
R_0list=np.logspace(1,4,20)

discretenesslist=[]
dimlist=[]
dimstd=[]
for R_0 in R_0list:
    interval_centre=[R_0+L/2,0]
    for i in range(d-2):
        interval_centre.append(0)

    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L/2)**(1-d))
    densityads=N/V
    discreteness=densityads**(-1/d)
    discretenesslist.append(discreteness)

    Nrelsub=[]
    Nintervalsub=[]
    for count1 in range(20):
        position=ads.sprinkle(N,L,d,R_0,False,False)
        position_subset=filter_interval(position,L,interval_centre)
        relation=find_relation(position_subset)
        Nrel=sum(len(relation[i]) for i in range(len(relation)))
        Ninterval=len(position_subset)
        Nrelsub.append(Nrel)
        Nintervalsub.append(Ninterval)
        print(count1)
    Nrelmean=np.mean(Nrelsub)
    Nrelstd=np.std(Nrelsub)
    Nintervalmean=np.mean(Nintervalsub)
    Nintervalstd=np.std(Nintervalsub)
    fracmean=Nrelmean/Nintervalmean**2
    fracstd=np.sqrt((Nrelstd/Nintervalmean**2)**2+(Nintervalstd*2*Nrelmean/Nintervalmean**3)**2)
    upperdim=mm_dimension(fracmean+fracstd)
    lowerdim=mm_dimension(fracmean-fracstd)
    dimlist.append(np.mean([upperdim,lowerdim]))
    dimstd.append(np.std([upperdim,lowerdim]))
    print(R_0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
10.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
14.38449888287663
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20.6913808111479
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
29.76351441631318
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
42.81332398719393
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
61.58482110660264
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
88.58667904100822
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
127.42749857031335
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
183.29807108324357
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
263.6650898730358
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
379.26901907322497
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
545.5594781168514
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
784.7599703514607
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1128.8378916846884
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1623.776

In [30]:
plt.title(r'Discreteness length for interval at $R=R_0+L/2$;N=%s d=%s L=%s'%(N,d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel(r'$l_{discrete}$')
plt.xscale('log')
plt.plot(R_0list,discretenesslist)
plt.grid()
plt.show()

In [29]:
plt.title('Myrheim-Meyer dimension for interval at $R=R_0+L/2$;N=%s d=%s L=%s'%(N,d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel('Myrheim-Meyer dimension')
plt.xscale('log')
plt.errorbar(R_0list,dimlist,yerr=dimstd)
plt.grid()
plt.show()

In [48]:
#investigate full interval as function of d
N=10000
L=1
dlist=np.arange(3,9)
R_0=10000

discretenesslist=[]
dimlist=[]
dimstd=[]
for d in dlist:
    interval_centre=[R_0+L/2,0]
    for i in range(d-2):
        interval_centre.append(0)
    d=int(d)
    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L/2)**(1-d))
    densityads=N/V
    discreteness=densityads**(-1/d)
    discretenesslist.append(discreteness)

    Nrelsub=[]
    Nintervalsub=[]
    for count1 in range(10):
        position=ads.sprinkle(N,L,d,R_0,False,False)
        position_subset=filter_interval(position,L,interval_centre)
        relation=find_relation(position_subset)
        Nrel=sum(len(relation[i]) for i in range(len(relation)))
        Ninterval=len(position_subset)
        Nrelsub.append(Nrel)
        Nintervalsub.append(Ninterval)
    Nrelmean=np.mean(Nrelsub)
    Nrelstd=np.std(Nrelsub)
    Nintervalmean=np.mean(Nintervalsub)
    Nintervalstd=np.std(Nintervalsub)
    fracmean=Nrelmean/Nintervalmean**2
    fracstd=np.sqrt((Nrelstd/Nintervalmean**2)**2+(Nintervalstd*2*Nrelmean/Nintervalmean**3)**2)
    upperdim=mm_dimension(fracmean+fracstd)
    lowerdim=mm_dimension(fracmean-fracstd)
    dimlist.append(np.mean([upperdim,lowerdim]))
    dimstd.append(np.std([upperdim,lowerdim]))
    print(d)

3
4
5
6
7
8


In [51]:
plt.title(r'Discreteness length for interval at $R=R_0+L/2$;N=%s $R_0$=%s L=%s'%(N,R_0,L),fontsize=15)
plt.xlabel(r'AdS sprinkled dimension')
plt.ylabel(r'$l_{discrete}$')
plt.plot(dlist,discretenesslist)
plt.grid()
plt.show()


In [50]:
plt.title('Myrheim-Meyer dimension for interval at $R=R_0+L/2$;N=%s $R_0$=%s L=%s'%(N,R_0,L),fontsize=15)
plt.xlabel(r'AdS sprinkled dimension')
plt.ylabel('Myrheim-Meyer dimension')
plt.errorbar(dlist,dimlist,yerr=dimstd)
plt.grid()
plt.show()

In [64]:
#investigate half interval as function of R_0
N=2000
L=0.01
d=2
R_0list=np.logspace(-5,-1,20)

discretenesslist=[]
dimlist=[]
dimstd=[]
for R_0 in R_0list:
    interval_centre=[R_0,0]
    for i in range(d-2):
        interval_centre.append(0)
    d=int(d)
    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L/2)**(1-d))
    densityads=N/V
    discreteness=densityads**(-1/d)
    discretenesslist.append(discreteness)

    Nrelsub=[]
    Nintervalsub=[]
    for count1 in range(20):
        position=ads.sprinkle(N,L,d,R_0,False,True)
        position_subset=filter_interval(position,L,interval_centre)
        relation=find_relation(position_subset)
        Nrel=sum(len(relation[i]) for i in range(len(relation)))
        Ninterval=len(position_subset)
        Nrelsub.append(Nrel)
        Nintervalsub.append(Ninterval)
        print(count1)
    Nrelmean=np.mean(Nrelsub)
    Nrelstd=np.std(Nrelsub)
    Nintervalmean=np.mean(Nintervalsub)
    Nintervalstd=np.std(Nintervalsub)
    fracmean=Nrelmean/Nintervalmean**2
    fracstd=np.sqrt((Nrelstd/Nintervalmean**2)**2+(Nintervalstd*2*Nrelmean/Nintervalmean**3)**2)
    upperdim=mm_dimension(fracmean+fracstd)
    lowerdim=mm_dimension(fracmean-fracstd)
    dimlist.append(np.mean([upperdim,lowerdim]))
    dimstd.append(np.std([upperdim,lowerdim]))
    print(R_0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1.623776739188721e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2.6366508987303556e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
4.281332398719396e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
6.951927961775606e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.00011288378916846884
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.00018329807108324357
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.00029763514416313193
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.0004832930238571752
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.0007847599703514606
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.0012742749857031334
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.00206913808111479
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.003359818286283781
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.005455594781168515
0
1
2
3


In [72]:
plt.title(r'Discreteness length for interval at $R=R_0+L/2$;N=%s $d$=%i L=%s'%(N,d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel(r'$l_{discrete}$')
plt.xscale('log')
plt.plot(R_0list,discretenesslist)
plt.grid()
plt.show()

In [73]:
plt.title('Myrheim-Meyer dimension for interval at $R=R_0+L/2$;N=%s $d$=%i L=%s'%(N,d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel('Myrheim-Meyer dimension')
plt.xscale('log')
plt.errorbar(R_0list,dimlist,yerr=dimstd)
plt.grid()
plt.show()

In [58]:
#investigate half interval as function of d
N=10000
L=1
dlist=np.arange(3,7)
R_0=0.0001

discretenesslist=[]
dimlist=[]
dimstd=[]
for d in dlist:
    interval_centre=[R_0,0]
    for i in range(d-2):
        interval_centre.append(0)
    d=int(d)
    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L/2)**(1-d))
    densityads=N/V
    discreteness=densityads**(-1/d)
    discretenesslist.append(discreteness)

    Nrelsub=[]
    Nintervalsub=[]
    for count1 in range(20):
        position=ads.sprinkle(N,L,d,R_0,False,True)
        position_subset=filter_interval(position,L,interval_centre)
        relation=find_relation(position_subset)
        Nrel=sum(len(relation[i]) for i in range(len(relation)))
        Ninterval=len(position_subset)
        Nrelsub.append(Nrel)
        Nintervalsub.append(Ninterval)
        print(count1)
    Nrelmean=np.mean(Nrelsub)
    Nrelstd=np.std(Nrelsub)
    Nintervalmean=np.mean(Nintervalsub)
    Nintervalstd=np.std(Nintervalsub)
    fracmean=Nrelmean/Nintervalmean**2
    fracstd=np.sqrt((Nrelstd/Nintervalmean**2)**2+(Nintervalstd*2*Nrelmean/Nintervalmean**3)**2)
    upperdim=mm_dimension(fracmean+fracstd)
    lowerdim=mm_dimension(fracmean-fracstd)
    dimlist.append(np.mean([upperdim,lowerdim]))
    dimstd.append(np.std([upperdim,lowerdim]))
    print(d)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
6


In [63]:
plt.title(r'Discreteness length for interval at $R=R_0$;N=%s $R_0$=%s L=%s'%(N,R_0,L),fontsize=15)
plt.xlabel(r'AdS sprinkled dimension')
plt.ylabel(r'$l_{discrete}$')
plt.plot(dlist,discretenesslist)
plt.grid()
plt.show()


In [62]:
plt.title('Myrheim-Meyer dimension for interval at $R=R_0$;N=%s $R_0$=%s L=%s'%(N,R_0,L),fontsize=15)
plt.xlabel(r'AdS sprinkled dimension')
plt.ylabel('Myrheim-Meyer dimension')
plt.errorbar(dlist,dimlist,yerr=dimstd)
plt.grid()
plt.show()

In [16]:
Nlist=np.linspace(100,1000,20)
L=1
d=2
R_0=0.0001
interval_centre=[R_0,0]
for i in range(d-2):
    interval_centre.append(0)
discretenesslist=[]
Nintervallist=[]
Nintervalstd=[]
lengthlist=[]
lengthstd=[]
for N in Nlist:
    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L/2)**(1-d))
    densityads=N/V
    discreteness=densityads**(-1/d)
    discretenesslist.append(discreteness)
    Nintervalsub=[]
    lengthsub=[]
    for count1 in range(20):
        position=ads.sprinkle(N,L,d,R_0,False,True)
        position_subset=filter_interval(position,L,interval_centre)
        link=find_link(position_subset)
        longest=longest_chain(link)
        length=len(longest)
        lengthsub.append(length)
        Ninterval=len(position_subset)
        Nintervalsub.append(Ninterval)
        print(count1)
    Nintervallist.append(np.mean(Nintervalsub))
    Nintervalstd.append(np.std(Nintervalsub))
    lengthlist.append(np.mean(lengthsub))
    lengthstd.append(np.std(lengthsub))
    print(N)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
100.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
147.3684210526316
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
194.73684210526318
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
242.10526315789474
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
289.47368421052636
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
336.8421052631579
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
384.2105263157895
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
431.5789473684211
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
478.94736842105266
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
526.3157894736842
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
573.6842105263158
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
621.0526315789474
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
668.421052631579
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
715.7894736842105
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
763.157

In [17]:
fit=op.curve_fit(fitting_func,Nintervallist,lengthlist,p0=[2,d-1],sigma=lengthstd)
print('m2=',fit[0][0],',d=',fit[0][1])
plt.title(r'Longest chain length scaling against $\left\langle N_{interval}\right\rangle$; d=%s, $R_0$=%s, L=%s'%(d,R_0,L),fontsize=15)
plt.xlabel(r'$\left\langle N_{interval}\right\rangle$')
plt.ylabel('Length of longest chain')
plt.plot(Nintervallist,fitting_func(Nintervallist,*fit[0]))
plt.errorbar(Nintervallist,lengthlist,yerr=lengthstd,xerr=Nintervalstd)
plt.grid()
plt.show()

m2= 1.4722977816761358 ,d= 1.0961104338316796


In [82]:
Nlist=np.linspace(100,1000,15)
L=1
d=3
R_0list=np.logspace(-7,-4,25)
dlist=[]
dstd=[]
mlist=[]
mstd=[]
discretenesslist=[]
for R_0 in R_0list:
    interval_centre=[R_0,0]
    for i in range(d-2):
        interval_centre.append(0)
    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L/2)**(1-d))
    Nintervallist=[]
    # Nintervalstd=[]
    lengthlist=[]
    lengthstd=[]
    for N in Nlist:
        densityads=N/V
        discreteness=densityads**(-1/d)
        discretenesslist.append(discreteness)
        Nintervalsub=[]
        lengthsub=[]
        for count1 in range(20):
            position=ads.sprinkle(N,L,d,R_0,False,True)
            position_subset=filter_interval(position,L,interval_centre)
            link=find_link(position_subset)
            longest=longest_chain(link)
            length=len(longest)
            lengthsub.append(length)
            Ninterval=len(position_subset)
            Nintervalsub.append(Ninterval)
            print(count1)
        Nintervallist.append(np.mean(Nintervalsub))
        # Nintervalstd.append(np.std(Nintervalsub))
        lengthlist.append(np.mean(lengthsub))
        lengthstd.append(np.std(lengthsub))
        print('N=',N)
    fit=op.curve_fit(fitting_func,Nintervallist,lengthlist,p0=[2,d-1])
    plt.errorbar(Nintervallist,lengthlist,yerr=lengthstd)
    plt.plot(Nintervallist,fitting_func(Nintervallist,*fit[0]))
    plt.show()
    mlist.append(fit[0][0])
    mstd.append(fit[1][0,0])
    dlist.append(fit[0][1])
    dstd.append(fit[1][1,1])

    print('R_0=',R_0)

0
1
N= 800.0
0
1
N= 814.2857142857143
0
1
N= 828.5714285714286
0
1
N= 842.8571428571429
0
1
N= 857.1428571428571
0
1
N= 871.4285714285714
0
1
N= 885.7142857142858
0
1
N= 900.0
0
1
N= 914.2857142857143
0
1
N= 928.5714285714286
0
1
N= 942.8571428571429
0
1
N= 957.1428571428571
0
1
N= 971.4285714285714
0
1
N= 985.7142857142858
0
1
N= 1000.0
R_0= 1e-07
0
1
N= 800.0
0
1
N= 814.2857142857143
0
1
N= 828.5714285714286
0
1
N= 842.8571428571429
0
1
N= 857.1428571428571
0
1
N= 871.4285714285714
0
1
N= 885.7142857142858


KeyboardInterrupt: 

In [83]:
plt.title('Longest chain scaling of dimension with half interval; d=%s, L=%s'%(d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel('scaling dimension')
plt.xscale('log')
plt.grid()
plt.errorbar(R_0list,dlist,yerr=dstd)
plt.show()

ValueError: 'x' and 'y' must have the same size

In [69]:
plt.title(r'Longest chain scaling of $m_d$ with half interval; d=%s, L=%s'%(d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel(r'$m_d$')
plt.xscale('log')
plt.grid()
plt.errorbar(R_0list,mlist,yerr=mstd)
plt.show()

In [84]:
print(dlist)

[1.3220647306423297]
